In [ ]:
# pip install sherpa_onnx,sounddevice

## 说话实时转写

In [ ]:
print("用户:  请输入你的问题", end='')
print('\r',end='')
print('-'*20,end='')
print('\r',end='')
print('用户：',end='')
print('您好呀,',end='', flush=True)
print('\r,', end='', flush=True)
# # print('1111', end='')
print('\r用户：您好呀'+' '*20,end='', flush=True)

In [ ]:
import sherpa_onnx
import sys,os
from kaldi.sherpa_onnx_model.stream_zipformer import microphone_endpoint
import re

#获取LargeModel所在的路径（事先将它添加到PYTHONPATH的环境变量中）
path= [content for content in sys.path if re.match('.+:.*LargeModel$', content)][0]
# path = "E:\LargeModel".replace('\\','/')
#获取stream_zipformer所在的文件路径
path = path+'/'+'kaldi/sherpa_onnx_model/stream_zipformer'
args_dict = {  
    'tokens': path+'/tokens.txt', 
    'encoder': path+'/encoder-epoch-99-avg-1.onnx',
    'decoder': path+'/decoder-epoch-99-avg-1.onnx',
    'joiner': path+'/joiner-epoch-99-avg-1.onnx',    
} 
sherpa_params ={
    'rule1_min_trailing_silence':3,#2.4,#完全静默超过这个时长视为端点,结束转写
    'rule2_min_trailing_silence':0.6,#1.2,已经解码过声音了，若静默超过这个时长，视为端点（输出一个空格）
    'provider':'cuda',#args.provider,
}
#检查设备，顺便输出recongnizer
check_recognizer= microphone_endpoint.main(args_dict, sherpa_params=sherpa_params)
# #录音转写
microphone_endpoint.main(args_dict,False, check_recognizer)

### API

In [5]:
import requests
url = 'http://127.0.0.1:9337/flag'
res = requests.post(url,json={})
res = eval(res.content.decode(encoding='utf-8'))[1]
res

NameError: name 'null' is not defined

## wav音频文件转写
#### 限制多，完全不如直接使用fast_whisper转写

In [ ]:
import sherpa_onnx
import sys
import re

#获取LargeModel所在的路径（事先将它添加到PYTHONPATH的环境变量中）
path= [content for content in sys.path if re.match('.+:.*LargeModel$', content)][0]
#获取stream_zipformer所在的文件路径
path = path+'/'+'kaldi/sherpa_onnx_model/stream_zipformer'

from kaldi.sherpa_onnx_model.stream_zipformer import decode_files
# import microphone_endpoint
args_dict = {  
    'tokens': path+'/tokens.txt',  
    'encoder': path+'/encoder-epoch-99-avg-1.onnx',  
    'decoder': path+'/decoder-epoch-99-avg-1.onnx' ,  
    'joiner': path+'/joiner-epoch-99-avg-1.onnx',    
    'num_threads':1,
    'decoding_method':"greedy_search",
    'max_active_paths':4,
    'lm':'',
    'lm_scale':0.1,
    'provider':'cuda',
    'hotwords_file':'',
    'hotwords_score':1.5,
    'blank_penalty':0.0} 

original_stdout =  sys.stdout
sys.stdout = open('log_maple.txt', 'w')
#必须是wave格式，单通道，16bit---decode_files.py 的267行更改通道数，297行更改采样率
sound_files=[path+'/test_wavs/0.wav',path+'/test_wavs/1.wav',path+'/test_wavs/2.wav',path+'/test_wavs/3.wav',path+'/test_wavs/8k.wav']
# sound_files=["E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\姬如千泷\素材\音频提取.wav"]

decode_files.main(args_dict, sound_files)
sys.stdout = original_stdout